In [1]:
import json
import os
from glob import glob

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
from google.colab import drive
drive.mount('/content/myDrive')

Mounted at /content/myDrive


In [ ]:
os.chdir('myDrive/MyDrive/ufru')

In [ ]:
!ls

public_data	 simple_answer.csv  train_data
public_data.zip  train_anwers.csv   train_data.zip


In [2]:
# !unzip public_data.zip

In [3]:
# !unzip train_data.zip

In [ ]:
y = pd.read_csv('train_anwers.csv', sep='\t')
print(y.dtypes)
y.set_index('name', inplace=True)
y.sort_values(by='name', axis=0, inplace=True)
y.head()

name      int64
target     bool
dtype: object


,target
name,
1,True
7,False
16,False
24,False
29,True


In [ ]:
raw_data = {}
for name in y.index:
    with open(f'train_data/{name}.json', 'r', encoding='utf-8') as inp:
        raw_data[name] = json.load(inp)

In [ ]:
players_matrix_as_dict = {}
for k in raw_data.keys():
    for i, player in enumerate(raw_data[k]['players']):
        if i >= 5:
            break
        player['target'] = int(y.loc[k])
        players_matrix_as_dict[f'{k}-{i}'] = player

In [ ]:
players_matrix = pd.DataFrame.from_dict(players_matrix_as_dict, orient='index')

In [ ]:
players_matrix

,player_slot,ability_targets,ability_upgrades_arr,ability_uses,actions,additional_units,assists,backpack_0,backpack_1,backpack_2,...,item_usage,purchase_tpscroll,actions_per_min,life_state_dead,purchase_ward_observer,lane_efficiency,lane_efficiency_pct,purchase_ward_sentry,purchase_gem,purchase_rapier
1-0,0,None,"[5175, 5173, 5173, 5174, 5175, 5176, 5175, 517...",None,None,None,17,218,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-1,1,None,"[5110, 5111, 641, 5111, 5111, 5113, 5110, 641,...",None,None,None,-5,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-2,2,None,"[5075, 5076, 5076, 5074, 5076, 5077, 5076, 507...",None,None,None,10,41,216,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-3,3,None,"[5395, 5397, 5395, 5397, -318, 5398, 5395, 539...",None,None,None,25,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-4,0,None,"[5140, 5138, 5138, 5140, 5138, 0, 5138, 219, 5...",None,None,None,19,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35552-0,0,None,"[5260, 1645, 5260, 5259, 5259, 5262, 5259, 525...",None,None,None,7,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35552-1,1,None,"[5164, 0, 5164, 0, 5164]",None,None,None,-1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35552-2,0,None,"[5341, 5340, 5341, 5340, 5339, 2353, 5341, 533...",None,None,None,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35552-3,3,None,"[5486, 1694, 5486, 5488, 5486, 5487]",None,None,None,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
num_cols = [col for col in players_matrix.columns
            if players_matrix[col].dtype in ['int64', 'float64']]

cat_cols = [col for col in players_matrix.columns if
            players_matrix[col].dtype == "object"]

bool_cols = [col for col in players_matrix.columns if
             players_matrix[col].dtype == 'bool']

In [ ]:
corr = players_matrix[num_cols].corr()['target'].sort_values(ascending=False)
corr

target                1.000000
tower_damage          0.399670
kda                   0.382888
purchase_gem          0.353553
towers_killed         0.329270
                        ...   
leaver_status        -0.105042
life_state_dead      -0.154737
deaths               -0.285179
purchase_rapier      -1.000000
necronomicon_kills         NaN
Name: target, Length: 78, dtype: float64

In [ ]:
players_matrix.shape

(35845, 136)

In [ ]:
na_count = players_matrix[num_cols].isna().sum().sort_values(ascending=False)
na_count

purchase_rapier           35842
purchase_gem              35838
purchase_ward_sentry      35701
purchase_ward_observer    35681
purchase_tpscroll         35659
                          ...  
start_time                    0
duration                      0
cluster                       0
lobby_type                    0
xp_per_min                    0
Length: 78, dtype: int64

In [ ]:
num_cols_without_na = na_count[na_count <= players_matrix.shape[0] - 100].index

In [ ]:
corr = players_matrix[num_cols_without_na].corr()['target'].sort_values(ascending=False)
len(corr)

76

In [ ]:
important_num_player_cols = corr[np.abs(corr).sort_values(ascending=False) >= 0.1].index[1:]
important_num_player_cols # важные числовые признаки player 

Index(['tower_damage', 'kda', 'towers_killed', 'tower_kills', 'gold',
       'kills_per_min', 'assists', 'kills', 'roshan_kills', 'net_worth',
       'roshans_killed', 'hero_kills', 'gold_per_min', 'lane_efficiency_pct',
       'rune_pickups', 'total_gold', 'courier_kills', 'gold_spent',
       'actions_per_min', 'lane_kills', 'total_xp', 'purchase_tpscroll',
       'xp_per_min', 'pings', 'camps_stacked', 'leaver_status',
       'life_state_dead', 'deaths'],
      dtype='object')

In [ ]:
X_dict = {}
for k in raw_data.keys():
    players_in_game = {}
    for i, player in enumerate(raw_data[k]['players']):
        for col in important_num_player_cols:
            if col not in player:
                players_in_game[f'{col}-{i}'] = None
            else:
                players_in_game[f'{col}-{i}'] = player[col]
    players_in_game['target'] = int(y.loc[k])
    X_dict[k] = players_in_game

In [ ]:
X = pd.DataFrame.from_dict(X_dict, orient='index')
X

,tower_damage-0,kda-0,towers_killed-0,tower_kills-0,gold-0,kills_per_min-0,assists-0,kills-0,roshan_kills-0,net_worth-0,...,lane_kills-9,total_xp-9,purchase_tpscroll-9,xp_per_min-9,pings-9,camps_stacked-9,leaver_status-9,life_state_dead-9,deaths-9,target
1,484.0,1.0,NaN,NaN,639.0,-0.009158,17.0,3.0,NaN,7014.0,...,NaN,12295.0,NaN,0.0,NaN,NaN,0.0,NaN,8.0,1
7,23.0,0.0,NaN,NaN,2576.0,0.014151,22.0,6.0,NaN,17151.0,...,NaN,23812.0,NaN,822.0,NaN,NaN,0.0,NaN,4.0,0
16,0.0,1.0,NaN,NaN,662.0,0.116788,-3.0,0.0,NaN,8277.0,...,NaN,8637.0,NaN,578.0,NaN,NaN,0.0,NaN,4.0,0
24,0.0,1.0,NaN,NaN,2.0,0.115340,17.0,0.0,NaN,7002.0,...,NaN,36278.0,NaN,854.0,NaN,NaN,0.0,NaN,7.0,0
29,240.0,1.0,NaN,NaN,1037.0,0.090361,14.0,4.0,NaN,12812.0,...,NaN,25276.0,NaN,234.0,NaN,NaN,0.0,NaN,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
35361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
35394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
35457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
X_train = X[X.columns[:-1]]
y_train = X['target']

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

model = GradientBoostingClassifier()
num_transformer = SimpleImputer(strategy="median")

# preprocessor = ColumnTransformer(transformers=[
#     ('numeric', num_transformer)
# ])

my_pipeline = Pipeline(steps=[
    ('preprocessor', num_transformer),
    ('model', model)
])

In [ ]:
cross_val_score(my_pipeline, X_train, y_train, scoring='f1')

array([0.98175182, 0.9848393 , 0.97949337, 0.97652017, 0.79281902])

In [ ]:
raw_test_data = {}
for _, _, files in os.walk('public_data'):
    for file in files:
        with open(f'public_data/{file}', 'r', encoding='utf-8') as inp:
            raw_test_data[int(file.split('.')[0])] = json.load(inp)

In [ ]:
X_test_dict = {}
for k in raw_test_data.keys():
    players_in_game = {}
    for i, player in enumerate(raw_test_data[k]['players']):
        for col in important_num_player_cols:
            if col not in player:
                players_in_game[f'{col}-{i}'] = None
            else:
                players_in_game[f'{col}-{i}'] = player[col]
    X_test_dict[k] = players_in_game

In [ ]:
X_test = pd.DataFrame.from_dict(X_test_dict, orient='index')
X_test

,tower_damage-0,kda-0,towers_killed-0,tower_kills-0,gold-0,kills_per_min-0,assists-0,kills-0,roshan_kills-0,net_worth-0,...,actions_per_min-9,lane_kills-9,total_xp-9,purchase_tpscroll-9,xp_per_min-9,pings-9,camps_stacked-9,leaver_status-9,life_state_dead-9,deaths-9
10013,2142,2,NaN,NaN,476,0.185886,13,9,NaN,22351,...,NaN,NaN,43187,NaN,892,NaN,NaN,0,NaN,6
10020,648,3,NaN,NaN,1049,0.225950,27,11,NaN,13849,...,NaN,NaN,45129,NaN,927,NaN,NaN,0,NaN,13
10037,25,0,NaN,NaN,297,0.076142,7,2,NaN,19127,...,NaN,NaN,45257,NaN,1723,NaN,NaN,0,NaN,2
10059,1080,1,NaN,NaN,1296,0.078782,28,5,NaN,9631,...,NaN,NaN,54708,NaN,862,NaN,NaN,0,NaN,4
10113,5172,6,NaN,NaN,3469,0.311015,20,12,NaN,23649,...,NaN,NaN,32410,NaN,840,NaN,NaN,0,NaN,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9808,0,0,NaN,NaN,612,0.031024,3,1,NaN,4067,...,NaN,NaN,19630,NaN,609,NaN,NaN,0,NaN,1
9860,5234,3,NaN,NaN,2356,0.221198,22,12,NaN,26716,...,NaN,NaN,36564,NaN,674,NaN,NaN,0,NaN,20
9862,1263,7,NaN,NaN,3196,0.084467,28,3,NaN,9621,...,NaN,NaN,10832,NaN,305,NaN,NaN,0,NaN,14
9866,4682,6,NaN,NaN,2326,0.053500,23,2,NaN,17006,...,NaN,NaN,11028,NaN,295,NaN,NaN,0,NaN,8


In [ ]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor', SimpleImputer(strategy='median')),
                ('model', GradientBoostingClassifier())])

In [ ]:
answer = pd.DataFrame()
answer['name'] = X_test.index
answer['target'] = my_pipeline.predict(X_test)

In [ ]:
answer.to_csv('simple_answer.csv', sep='\t', index=False)

In [ ]:
answer

,name,target
0,10013,0
1,10020,0
2,10037,0
3,10059,0
4,10113,1
...,...,...
995,9808,0
996,9860,1
997,9862,1
998,9866,1
